In [0]:
# imports pytorch
import json
import re
import torch
# imports the torch_xla package
# import torch_xla
# import torch_xla.core.xla_model as xm
# import torch_xla.distributed.xla_multiprocessing as xmp
# install
!pip install pytorch-pretrained-bert pytorch-nlp
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
# import parallel loader
# import torch_xla.distributed.parallel_loader as pl
import time
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, DistributedSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification, BertForQuestionAnswering
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

In [0]:
from scipy.special import softmax

In [0]:
# import selection binary models
selection_model = BertForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=2)
# load test model parameters
selection_path = F"/content/drive/My Drive/HydraNet/RetrainModels/SelectionClause/retrain_selectionclause_classifier_epoch_3.pt"
selection_model.load_state_dict(torch.load(selection_path))
# selection_model.to(test_device)

In [0]:
# import aggregation mullti-class models
aggregation_model = BertForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=6)
# load test model parameters
aggregation_path = F"/content/drive/My Drive/HydraNet/RetrainModels/AggregationOperator/retrain_aggregation_operator_classifier_epoch_3.pt"
aggregation_model.load_state_dict(torch.load(aggregation_path))
# aggregation_model.to(test_device)

In [0]:
# import wherenum mullti-class models
wherenum_model = BertForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=5)
# load test model parameters
wherenum_path = F"/content/drive/My Drive/HydraNet/RetrainModels/NumWhereConditions/retrain_wherenum_classifier_epoch_3.pt"
wherenum_model.load_state_dict(torch.load(wherenum_path))
# wherenum_model.to(test_device)

In [0]:
# import isin_relevance binary models
isinrelevance_model = BertForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=2)
# load test model parameters
isinrelevance_path = F"/content/drive/My Drive/HydraNet/RetrainModels/IsInRelevanceClause/retrain_isinrelevanceclause_classifier_epoch_3.pt"
isinrelevance_model.load_state_dict(torch.load(isinrelevance_path))
# isinrelevance_model.to(test_device)

In [0]:
# import isin_where binary models
isinwhere_model = BertForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=2)
# load test model parameters
isinwhere_path = F"/content/drive/My Drive/HydraNet/RetrainModels/IsInWhereClause/retrain_isinwhereclause_classifier_epoch_3.pt"
isinwhere_model.load_state_dict(torch.load(isinwhere_path))
# isinwhere_model.to(test_device)

In [0]:
# import where operator multi-class models
whereoperator_model = BertForSequenceClassification.from_pretrained("bert-large-uncased", num_labels=3)
# load test model parameters
whereoperator_path = F"/content/drive/My Drive/HydraNet/RetrainModels/ConditionOperator/retrain_condition_operator_classifier_epoch_3.pt"
whereoperator_model.load_state_dict(torch.load(whereoperator_path))
# whereoperator_model.to(test_device)

In [0]:
# import where value question-answering models
wherevalue_model = BertForQuestionAnswering.from_pretrained("bert-large-uncased")
# load test model parameters
wherevalue_path = F"/content/drive/My Drive/HydraNet/RetrainModels/WhereValue/retrain_where_value_model_epoch_3.pt"
wherevalue_model.load_state_dict(torch.load(wherevalue_path))
# wherevalue_model.to(test_device)

In [0]:
# Input questions and table headers as well as table type
input_question = 'what is the total revenue for apple in canada when the profit is more than 300'
input_columns = ['Region', 'Fruit', 'Amount (kilo) weight', 'Salesperson', 'Customer Type', 'Revenue (dollar)', 'Profit (dollar)']
input_types = ['text', 'text', 'real', 'text', 'text', 'real', 'real']
MAX_LEN = 64
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)
aggregation_operator_list = ["NAN", "max", "min", "nunique", "sum", "mean"]
condition_operator_list = ["=", ">", "<"]
table_name = 'Fruits'

Form the [CLS] + type + column + [SEP] + question + [SEP] input

In [0]:
def prepare_input(question, columns, types, max_len, bert_tokenizer):
  list_of_str = []
  for i in range(len(columns)):
    input_str = '[CLS] ' + str(types[i]) + ' ' + str(columns[i]) + ' [SEP] ' + str(question) + ' [SEP]'
    list_of_str.append(input_str)
  test_tokenized = [bert_tokenizer.tokenize(sentence) for sentence in list_of_str]
  test_input_ids = [bert_tokenizer.convert_tokens_to_ids(x) for x in test_tokenized]
  test_input_ids = pad_sequences(test_input_ids, maxlen=max_len, dtype="long", truncating="post", padding="post")
  test_attention_masks = []
  test_segment_ids = []
  for seq in test_input_ids:
    seq_mask = [float(i > 0) for i in seq]
    test_attention_masks.append(seq_mask)
    seq_segment = [0] * len(seq)
    seq_np = np.array(seq)
    temp = np.nonzero(seq_np == 102)
    if len(temp) > 0:
      if len(temp[0]) > 0:
        index = temp[0][0]
        seq_segment[index+1:] = [1] * (len(seq) - (index + 1))
    test_segment_ids.append(seq_segment)
  return test_tokenized, test_input_ids, test_attention_masks, test_segment_ids 

Prediction Selection Clause. Give the input columns list, ['Region', 'Fruit', 'Amount Sold', 'Salesperson', 'Customer Type', 'Revenue', 'Profit'], find the index of selection column in the columns list

In [0]:
def prediction_selection_clause(selection_model, test_input_ids, test_attention_masks, test_segment_ids):
  # put the tensors into tpu
  # selection_model.to(test_device)
  test_input_ids = torch.tensor(test_input_ids)
  test_attention_masks = torch.tensor(test_attention_masks)
  test_segment_ids = torch.tensor(test_segment_ids)
  test_input_ids_tpu = test_input_ids
  test_attention_masks_tpu = test_attention_masks
  test_segment_ids_tpu = test_segment_ids
  selection_model.eval()
  with torch.no_grad():
    logits = selection_model(input_ids=test_input_ids_tpu, token_type_ids=test_segment_ids_tpu, attention_mask=test_attention_masks_tpu)
  logits = logits.detach().cpu().numpy()
  sigmoid_logits = [1/(1+np.exp(-1 * each_logit)) for each_logit in logits]
  sigmoid_logits_1 = []
  for each_row in sigmoid_logits:
    # only care about the class that labels as 1
    sigmoid_logits_1.append(each_row[1])
  max_index = np.argmax(sigmoid_logits_1)
  return max_index, sigmoid_logits_1

In [0]:
def prediction_aggregation_operator(aggregation_model, test_input_ids, test_attention_masks, test_segment_ids):
  # aggregation_model.to(test_device)
  test_input_ids_agg = [test_input_ids]
  test_input_ids_agg = torch.tensor(test_input_ids_agg)
  test_input_ids_agg_tpu = test_input_ids_agg
  test_attention_masks_agg = [test_attention_masks]
  test_attention_masks_agg = torch.tensor(test_attention_masks_agg)
  test_attention_masks_agg_tpu = test_attention_masks_agg
  test_segment_ids_agg = [test_segment_ids]
  test_segment_ids_agg = torch.tensor(test_segment_ids_agg)
  test_segment_ids_agg_tpu = test_segment_ids_agg
  aggregation_model.eval()
  with torch.no_grad():
    logits = aggregation_model(input_ids=test_input_ids_agg_tpu, token_type_ids=test_segment_ids_agg_tpu, attention_mask=test_attention_masks_agg_tpu)
  logits = logits.detach().cpu().numpy()
  sigmoid_logits = [1/(1+np.exp(-1 * each_logit)) for each_logit in logits]
  max_index = np.argmax(sigmoid_logits[0])
  return max_index, sigmoid_logits[0]

The following two functions predict the num of where clauses

In [0]:
def prediction_wherenum(wherenum_model, test_input_ids, test_attention_masks, test_segment_ids):
  # put the tensors into tpu
  # wherenum_model.to(test_device)
  test_input_ids = torch.tensor(test_input_ids)
  test_attention_masks = torch.tensor(test_attention_masks)
  test_segment_ids = torch.tensor(test_segment_ids)
  test_input_ids_tpu = test_input_ids
  test_attention_masks_tpu = test_attention_masks
  test_segment_ids_tpu = test_segment_ids
  wherenum_model.eval()
  with torch.no_grad():
    logits = wherenum_model(input_ids=test_input_ids_tpu, token_type_ids=test_segment_ids_tpu, attention_mask=test_attention_masks_tpu)
  logits = logits.detach().cpu().numpy()
  return logits

In [0]:
def prediction_isinrelevance(isinrelevance_model, test_input_ids, test_attention_masks, test_segment_ids):
  # put the tensors into tpu
  # isinrelevance_model.to(test_device)
  test_input_ids = torch.tensor(test_input_ids)
  test_attention_masks = torch.tensor(test_attention_masks)
  test_segment_ids = torch.tensor(test_segment_ids)
  test_input_ids_tpu = test_input_ids
  test_attention_masks_tpu = test_attention_masks
  test_segment_ids_tpu = test_segment_ids
  isinrelevance_model.eval()
  with torch.no_grad():
    logits = isinrelevance_model(input_ids=test_input_ids_tpu, token_type_ids=test_segment_ids_tpu, attention_mask=test_attention_masks_tpu)
  logits = logits.detach().cpu().numpy()
  sigmoid_logits = [1/(1+np.exp(-1 * each_logit)) for each_logit in logits]
  sigmoid_logits_1 = []
  for each_row in sigmoid_logits:
    # only care about the class that labels as 1
    sigmoid_logits_1.append(each_row[1])
  return sigmoid_logits_1

Predict if a column is in where clause

In [0]:
def prediction_isinwhere(isinwhere_model, test_input_ids, test_attention_masks, test_segment_ids):
  # put the tensors into tpu
  # isinwhere_model.to(test_device)
  test_input_ids = torch.tensor(test_input_ids)
  test_attention_masks = torch.tensor(test_attention_masks)
  test_segment_ids = torch.tensor(test_segment_ids)
  test_input_ids_tpu = test_input_ids
  test_attention_masks_tpu = test_attention_masks
  test_segment_ids_tpu = test_segment_ids
  isinwhere_model.eval()
  with torch.no_grad():
    logits = isinwhere_model(input_ids=test_input_ids_tpu, token_type_ids=test_segment_ids_tpu, attention_mask=test_attention_masks_tpu)
  logits = logits.detach().cpu().numpy()
  sigmoid_logits = [1/(1+np.exp(-1 * each_logit)) for each_logit in logits]
  sigmoid_logits_1 = []
  for each_row in sigmoid_logits:
    # only care about the class that labels as 1
    sigmoid_logits_1.append(each_row[1])
  return sigmoid_logits_1

In [0]:
def prediction_wherecolumns(probability, numwhere):
  array_probability = np.array(probability)
  return array_probability.argsort()[-numwhere:][::-1]

Given the where condition list, ['=', '>', '<', 'OP'], find the condition operator index for where columns. (Ignore 'OP', because no questions have 'OP' as the condition operator for WIKISQL)

In [0]:
def prediction_where_operator(whereoperator_model, test_input_ids, test_attention_masks, test_segment_ids):
  # whereoperator_model.to(test_device)
  test_input_ids_whereoperator = test_input_ids
  test_input_ids_whereoperator = torch.tensor(test_input_ids_whereoperator)
  test_input_ids_whereoperator_tpu = test_input_ids_whereoperator
  test_attention_masks_whereoperator = test_attention_masks
  test_attention_masks_whereoperator = torch.tensor(test_attention_masks_whereoperator)
  test_attention_masks_whereoperator_tpu = test_attention_masks_whereoperator
  test_segment_ids_whereoperator = test_segment_ids
  test_segment_ids_whereoperator = torch.tensor(test_segment_ids_whereoperator)
  test_segment_ids_whereoperator_tpu = test_segment_ids_whereoperator
  whereoperator_model.eval()
  with torch.no_grad():
    logits = whereoperator_model(input_ids=test_input_ids_whereoperator_tpu, token_type_ids=test_segment_ids_whereoperator_tpu, attention_mask=test_attention_masks_whereoperator_tpu)
  logits = logits.detach().cpu().numpy()
  sigmoid_logits = [1/(1+np.exp(-1 * each_logit)) for each_logit in logits]
  max_index = np.argmax(sigmoid_logits, axis=1)
  return max_index, sigmoid_logits

Predict Where Value

In [0]:
def prediction_where_value(wherevalue_model, test_input_ids, test_attention_masks, test_segment_ids):
  # wherevalue_model.to(test_device)
  test_input_ids_whereoperator = test_input_ids
  test_input_ids_whereoperator = torch.tensor(test_input_ids_whereoperator)
  test_input_ids_whereoperator_tpu = test_input_ids_whereoperator
  test_attention_masks_whereoperator = test_attention_masks
  test_attention_masks_whereoperator = torch.tensor(test_attention_masks_whereoperator)
  test_attention_masks_whereoperator_tpu = test_attention_masks_whereoperator
  test_segment_ids_whereoperator = test_segment_ids
  test_segment_ids_whereoperator = torch.tensor(test_segment_ids_whereoperator)
  test_segment_ids_whereoperator_tpu = test_segment_ids_whereoperator
  wherevalue_model.eval()
  with torch.no_grad():
    start_logits, end_logits = wherevalue_model(input_ids=test_input_ids_whereoperator_tpu, token_type_ids=test_segment_ids_whereoperator_tpu, attention_mask=test_attention_masks_whereoperator_tpu)
  start_logits = start_logits.detach().cpu().numpy()
  end_logits = end_logits.detach().cpu().numpy()
  sigmoid_start_logits = [1/(1+np.exp(-1 * each_logit)) for each_logit in start_logits]
  sigmoid_end_logits = [1/(1+np.exp(-1 * each_logit)) for each_logit in end_logits]
  start_index = np.argmax(sigmoid_start_logits, axis=1)
  end_index = np.argmax(sigmoid_end_logits, axis=1)
  return start_index, end_index, sigmoid_start_logits, sigmoid_end_logits

In [0]:
def final_function(input_question, input_columns, input_types, max_len, tokenizer, aggregation_operator_list, condition_operator_list, table_name, selection_model, aggregation_model, wherenum_model, isinrelevance_model, isinwhere_model, whereoperator_model, wherevalue_model):
  current_test_tokenized, current_test_input_ids, current_test_attention_masks, current_test_segment_ids = prepare_input(question=input_question, columns=input_columns, types=input_types, max_len=max_len, bert_tokenizer=tokenizer)
  selection_index, selection_sigmoid = prediction_selection_clause(selection_model=selection_model, test_input_ids=current_test_input_ids, test_attention_masks=current_test_attention_masks, test_segment_ids=current_test_segment_ids)
  aggregation_index, aggregation_sigmoid = prediction_aggregation_operator(aggregation_model=aggregation_model, test_input_ids=current_test_input_ids[selection_index], test_attention_masks=current_test_attention_masks[selection_index], test_segment_ids=current_test_segment_ids[selection_index])
  wherenum_probability = prediction_wherenum(wherenum_model=wherenum_model, test_input_ids=current_test_input_ids, test_attention_masks=current_test_attention_masks, test_segment_ids=current_test_segment_ids)
  isinrelevance_probability = prediction_isinrelevance(isinrelevance_model=isinrelevance_model, test_input_ids=current_test_input_ids, test_attention_masks=current_test_attention_masks, test_segment_ids=current_test_segment_ids)
  wherenum_prediction = np.argmax(np.matmul(np.transpose(wherenum_probability), np.array(isinrelevance_probability)))
  isinwhere_probability = prediction_isinwhere(isinwhere_model=isinwhere_model, test_input_ids=current_test_input_ids, test_attention_masks=current_test_attention_masks, test_segment_ids=current_test_segment_ids)
  where_columns = prediction_wherecolumns(isinwhere_probability, wherenum_prediction)
  whereoperator_index, whereoperator_sigmoid = prediction_where_operator(whereoperator_model=whereoperator_model, test_input_ids=[current_test_input_ids[index] for index in where_columns], test_attention_masks=[current_test_attention_masks[index] for index in where_columns], test_segment_ids=[current_test_segment_ids[index] for index in where_columns])
  start_index, end_index, start_probability, end_probability = prediction_where_value(wherevalue_model=wherevalue_model, test_input_ids=[current_test_input_ids[index] for index in where_columns], test_attention_masks=[current_test_attention_masks[index] for index in where_columns], test_segment_ids=[current_test_segment_ids[index] for index in where_columns])
  # get actual where value
  where_value_list = []
  if len(where_columns) > 0:
    for i in range(len(where_columns)):
      current_tokens = current_test_tokenized[where_columns[i]]
      current_start_index = start_index[i]
      current_end_index = end_index[i]
      # need to take care of current_star_index == current_end_index == 0
      if current_start_index == 0 and current_end_index == 0:
        # get second largest from start_probability and end_probability
        second_start_probability = np.partition(start_probability[i].flatten(), -2)[-2]
        second_end_probability = np.partition(end_probability[i].flatten(), -2)[-2]
        first_start_probability = start_probability[i][current_start_index]
        first_end_probability = end_probability[i][current_end_index]
        # apply a threshold
        if (first_end_probability / second_end_probability < 10) and (first_start_probability / second_start_probability < 10):
          current_start_index = list(start_probability[i]).index(second_start_probability)
          current_end_index = list(end_probability[i]).index(second_end_probability)
      value_span = current_tokens[current_start_index: current_end_index+1]
      predicted_answer = ''
      for i, each_token in enumerate(value_span):
        if i == 0:
          if len(each_token) > 2:
            if each_token[0:2] == '##':
              predicted_answer = predicted_answer + each_token[2:]
            else:
              predicted_answer = predicted_answer + each_token
          else:
            predicted_answer = predicted_answer + each_token
        else:
          if len(each_token) <= 2:
            predicted_answer = predicted_answer + ' ' + each_token
          elif each_token[0:2] == '##':
            predicted_answer = predicted_answer + each_token[2:]
          else:
            predicted_answer = predicted_answer + ' ' + each_token
      where_value_list.append(predicted_answer)
  final_selection_column = input_columns[selection_index]
  final_aggregation_operator = aggregation_operator_list[aggregation_index]
  final_where_clauses = []
  if len(where_columns) > 0:
    for i in range(len(where_columns)):
      current_where_clause = ''
      current_where_clause = current_where_clause + input_columns[where_columns[i]] + ' ' + condition_operator_list[whereoperator_index[i]] + ' ' + where_value_list[i]
      final_where_clauses.append(current_where_clause)
  final_query = 'SELECT '
  if final_aggregation_operator == 'NAN' and input_types[selection_index] == 'real':
    final_query = final_query + final_selection_column + ' '
  elif final_aggregation_operator == 'NAN' and input_types[selection_index] == 'text':
    final_query = final_query + 'DISTINCT ' + '(' + final_selection_column + ') '
  else:
    final_query = final_query + final_aggregation_operator + '(' + final_selection_column + ') '
  final_query = final_query + 'FROM ' + table_name
  if len(final_where_clauses) > 0:
    final_query = final_query + ' WHERE'
    for j in range(len(final_where_clauses)):
      if where_value_list[j] != '[CLS]':
        final_query = final_query + ' ' + final_where_clauses[j] + ' AND'
  final_query = final_query[:-4]
  return final_query


In [0]:
# Input questions and table headers as well as table type
input_question = 'Who should I find, if I want to buy fruits in America? Or If I want to buy fruits in America, who should I find? '
input_columns = ['Region', 'Fruit', 'Amount (kilo) weight', 'Salesperson', 'Customer Type', 'Revenue (dollar)', 'Profit (dollar)']
input_types = ['text', 'text', 'real', 'text', 'text', 'real', 'real']
MAX_LEN = 64
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased', do_lower_case=True)
aggregation_operator_list = ["NAN", "max", "min", "count distinct", "sum", "mean"]
condition_operator_list = ["=", ">", "<"]
table_name = 'Fruits'

In [0]:
final_result= final_function(input_question, input_columns, input_types, MAX_LEN, tokenizer, aggregation_operator_list, condition_operator_list, table_name, selection_model, aggregation_model, wherenum_model, isinrelevance_model, isinwhere_model, whereoperator_model, wherevalue_model)